<a href="https://colab.research.google.com/github/AbdurNawaz/BlackJack-v0/blob/master/BlackJack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gym
import numpy as np

from collections import defaultdict
from plot_utils import plot_blackjack_values, plot_policy

In [0]:
env = gym.make('Blackjack-v0')

In [5]:
print(env.observation_space)
print(env.action_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))
Discrete(2)


In [9]:
for i_episode in range(7):
    state = env.reset()
    while True:
        print(state)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        if done:
            print('End game! Reward: ', reward)
            print('You won :)\n') if reward > 0 else print('You lost :(\n')
            break

(10, 1, False)
End game! Reward:  -1.0
You lost :(

(18, 3, False)
End game! Reward:  1.0
You won :)

(13, 3, False)
End game! Reward:  -1.0
You lost :(

(14, 10, False)
End game! Reward:  -1.0
You lost :(

(18, 5, True)
(15, 5, False)
End game! Reward:  1.0
You won :)

(12, 2, False)
(13, 2, False)
(19, 2, False)
End game! Reward:  0.0
You lost :(

(7, 7, False)
End game! Reward:  1.0
You won :)



In [14]:
env.step(1)

((26, 7, False), -1, True, {})

In [0]:
def generate_episode_from_limit_stochastic(bj_env):
  episode = []
  state = bj_env.reset()
  while True:
    probs = [0.8, 0.2] if state[0] > 18 else [0.2, 0.8]
    action = np.random.choice(np.arange(2), p=probs)
    next_state, reward, done, info = bj_env.step(action)
    episode.append((state, action, reward))
    state = next_state
    if done:
      break
  return episode

In [18]:
for i in range(3):
    print(generate_episode_from_limit_stochastic(env))

[((18, 4, False), 1, 0), ((19, 4, False), 0, 1.0)]
[((15, 6, True), 1, 0), ((15, 6, False), 0, -1.0)]
[((21, 10, True), 0, 1.0)]
